In [12]:
import pandas as pd
import pickle
file = open("df.pickle","rb")
df = pickle.load(file)
num_csv = int(max(df['Count']))+1
df.shape

(3787, 6)

In [13]:
def clean_df(df):
    df = df.fillna(0)
    df = df.reindex(sorted(df.columns), axis=1)
    df.rename(columns=dict(zip(df.columns,[str(int(c)) if not c[0].isalpha() else c for c in df.columns])),inplace=True)
    return df


df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_aantal = clean_df(df_aantal)


df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_relative = clean_df(df_relative)

In [28]:
# make a growth df
df_growth = df_aantal
temp_columns = df_growth.columns
import numpy as np
def calc_growth(line):
    # line = df_aantal[df_aantal['Gemeente']=='Tilburg'].values[0]
    # growth defined as today/yesterday -1
    # print(line)
    growth = [round(y/x-1,2) if x>4 else 0 for x,y in zip(line[:-1],line[1:-1])]
    # moving average over the growth
    smoothed_growth = [(x1+x2+x3+x4)/3 for x1,x2,x3,x4 in zip(growth,growth[1:],growth[2:],growth[3:])]
    # growth defined as today/4 days ago -1
    alternative_growth = [round(y/x-1,2) if x>4 else 0 for x,y in zip(line[:-1],line[3:-1])]
    result = [0]*(len(line)-len(alternative_growth)-1) + alternative_growth + [line[-1]]
    # print(len(line), len(growth), len(smoothed_growth),len(alternative_growth), len(result))
    return result

df_growth = pd.DataFrame.from_records(df_growth.apply(calc_growth,axis=1))
df_growth.rename(columns=dict(zip(df_growth.columns,temp_columns)),inplace=True)

In [52]:
import altair as alt
import pandas as pd
from vega_datasets import data
url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

columns = [str(day) for day in range(0, num_csv)]
slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="select", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend):
    maximum = max([max(df_growth[c]) for c in columns]) if legend == 'groei' else max(df[str(num_csv-1)])
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns),
        default = '0'
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        tooltip=[alt.Tooltip('properties.statnaam:N', title="Gemeente"), alt.Tooltip(legend+':Q', title=legend)],

        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='spectral', type='symlog',domain=[maximum,0])),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=300,
        height=400
    ).transform_filter(
        select_year
    )    
    return chart


In [53]:
alt.hconcat(
    serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief'),serve_chart(df_growth, 'groei')
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.HConcatChart(...)

In [35]:
# inspect individual cities
x = list(df_aantal[df_aantal['Gemeente']=="Tilburg"].values[0][:-1])
y = list(range(len(x)))
df = pd.DataFrame(x,y).reset_index()
print(df.columns)
df.rename(columns=dict(zip(df.columns,[str(c) for c in df.columns])),inplace=True)

alt.Chart(df).mark_point().encode(
    x='index:Q',
    y='0:Q',
)

Index(['index', 0], dtype='object')


alt.Chart(...)

In [33]:
input_dropdown = alt.binding_select(options=df['Gemeente'].unique())
selection = alt.selection_single(fields=['Origin'], bind=input_dropdown, name='Country of ')
color = alt.condition(selection,
                    alt.Color('Origin:N', legend=None),
                    alt.value('lightgray'))

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=color,
    tooltip='Name:N'
).add_selection(
    selection
)


KeyError: 'Gemeente'

In [65]:
input_dropdown = alt.binding_select(options=df['Gemeente'].unique())
selection = alt.selection_single(fields=['Origin'], bind=input_dropdown, name='Country of ')
ys = alt.condition(selection,
                    alt.Y('Origin:N'),
                    alt.value('lightgray'))

chart = alt.Chart(df_aantal).mark_point().encode(
).transform_fold(
    columns, as_=['day', 'values']
).transform_calculate(
    day='parseInt(datum.day)',
    legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
).encode(
    x='Horsepower:Q',
    y=ys,
    
).add_selection(
    select_year
)  
chart

SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.core.BindRadioSelect->options, validating 'type'

        array(["'s-Gravenhage", "'s-Hertogenbosch",
       '* Bij 112 personen is de woonplaats niet van bekend',
       '* Bij 137 personen is de woonplaats niet van bekend',
       '* postcode ontbreekt bij 37 pt en 1 woont in buitenland',
       '* postcode ontbreekt bij 58 pt en 1 woont in buitenland',
       '* postcode ontbreekt bij 70 pt en 1 woont in buitenland',
       '* postcode ontbreekt bij 84 pt en 1 woont in buitenland',
       '* postcode ontbreekt bij 89 pt en 1 woont in buitenland',
       'Aa en Hunze', 'Aalsmeer', 'Aalten', 'Achtkarspelen',
       'Alblasserdam', 'Albrandswaard', 'Alkmaar', 'Almelo', 'Almere',
       'Alphen aan den Rijn', 'Alphen-Chaam', 'Altena', 'Ameland',
       'Amersfoort', 'Amstelveen', 'Amsterdam', 'Apeldoorn', 'Appingedam',
       'Arnhem', 'Assen', 'Asten', 'Baarle-Nassau', 'Baarn',
       'Barendrecht', 'Barnev

In [92]:
import numpy as np
test = df_aantal.T
test.columns = test.iloc[-1]
test = test.iloc[:,4:-1]
columns = list(test.columns)
select_box = alt.binding_select(options=columns, name='column')
sel = alt.selection_single(fields=['column'], bind=select_box, init={'column': 'Haarlem'})

alt.Chart(df_aantal).transform_fold(
    columns,
    as_=['column', 'value']
).transform_filter(
    sel  
).mark_point().encode(
    x='Count:Q',
    y='value:Q',
).add_selection(
    sel
)

alt.Chart(...)

In [91]:
test

Gemeente,* postcode ontbreekt bij 37 pt en 1 woont in buitenland,* postcode ontbreekt bij 58 pt en 1 woont in buitenland,* postcode ontbreekt bij 70 pt en 1 woont in buitenland,* postcode ontbreekt bij 84 pt en 1 woont in buitenland,* postcode ontbreekt bij 89 pt en 1 woont in buitenland,Aa en Hunze,Aalsmeer,Aalten,Achtkarspelen,Alblasserdam,...,Zeewolde,Zeist,Zevenaar,Zoetermeer,Zoeterwoude,Zuidplas,Zundert,Zutphen,Zwartewaterland,Zwijndrecht
Count,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,2,0,0,0,...,1,2,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,2,0,0,0,...,1,3,0,0,0,0,2,0,0,0


In [76]:
test.iloc[-1]

Gemeente
's-Gravenhage                                              44
's-Hertogenbosch                                           45
* Bij 112 personen is de woonplaats niet van bekend         0
* Bij 137 personen is de woonplaats niet van bekend        99
* postcode ontbreekt bij 37 pt en 1 woont in buitenland     0
                                                           ..
Zundert                                                     7
Zutphen                                                     4
Zwartewaterland                                            15
Zwijndrecht                                                 6
Zwolle                                                     23
Name: 18, Length: 369, dtype: object